In [2]:
%pip install neuralforecast

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [3]:
%pip install pytorch-lightning pytorch-forecasting


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/821.1 kB ? eta -:--:--
   ---------------------------------------- 821.1/821.1 kB 5.9 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [lightning]
   ---------------------------------------- 0/2 [l

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from numpy.random import seed
from random import randrange

In [5]:
%pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load volatility data (RV)
rv_df = pd.read_csv("SP100_vol_ffill.csv", index_col="Date", parse_dates=True)

# Create lag features
lag_1 = rv_df.shift(1)
lag_5 = rv_df.rolling(5).mean().shift(1)
lag_22 = rv_df.rolling(22).mean().shift(1)

# Stack into long format
df_long = rv_df.stack().reset_index()
df_long.columns = ["date", "ticker", "RV"]
df_long["lag_1"] = lag_1.stack().reset_index(drop=True)
df_long["lag_5"] = lag_5.stack().reset_index(drop=True)
df_long["lag_22"] = lag_22.stack().reset_index(drop=True)

# Time index per stock
df_long["time_idx"] = df_long.groupby("ticker").cumcount()

# Drop missing (from lag calculations)
df_long = df_long.dropna().reset_index(drop=True)


Create the TimeSeriesDataSet

In [23]:
from pytorch_forecasting import TimeSeriesDataSet

dataset = TimeSeriesDataSet(
    df_long[["time_idx", "ticker", "date", "RV"]],
    time_idx="time_idx",
    target="RV",
    group_ids=["ticker"],
    max_encoder_length=30,
    max_prediction_length=1,
    static_categoricals=["ticker"],
    time_varying_unknown_reals=["RV"],  # only this!
    add_target_scales=True,
    add_relative_time_idx=False,  # MUST be false
)


In [24]:
train_loader = dataset.to_dataloader(train=True, batch_size=64, num_workers=4)
val_loader = dataset.to_dataloader(train=False, batch_size=64, num_workers=4)


In [25]:
from pytorch_forecasting import NBeats

model = NBeats.from_dataset(
    dataset,
    stack_types=["generic"],
    num_blocks=[3],
    num_block_layers=[2],
    widths=[256],
    learning_rate=1e-3,
)


AssertionError: The only variable as input should be the target which is part of time_varying_unknown_reals